In [11]:
from cmlreaders import CMLReader, get_data_index
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sb
import scipy
from scipy import stats
import pandas_to_pybeh as pb
from tqdm import tqdm_notebook as tn

In [7]:
BAI_high = 15
BAI_low = 8
BDI_high = 16
BDI_low = 10

sel_sub = pd.DataFrame([])
master = pd.read_excel(r'./220302_all_neuropsych.xlsx',engine="openpyxl") #read master spreadsheet of all RAM subjects demo/neuropsych data

sub_mat=list(np.zeros(len(master))*np.nan )
df = get_data_index('r1')

for sub in df['subject'].unique():
    num = int(sub[2:5])
    for i in master.index:
        sel = master.iloc[i]['Subject Number']
        if sel == num:
            sub_mat[i]=sub

master['subject']=sub_mat
master = master.dropna(subset=['BAI','BDI'])

In [29]:
df = get_data_index('r1')
events = []
list_df = pd.DataFrame([])

experiments = ['FR1']
lag_crp = []
for sub in master['subject']:
    try:
        for exp in experiments:
            localizations=df[(df['subject']==sub) & (df['experiment']==exp)]['localization'].unique()        

            for loc in localizations:
                montages = df[(df['subject']==sub) & (df['experiment']==exp) & (df['localization']==loc)]['montage'].unique()            

                for mont in montages: # iterate over montages            
                    data = df[np.logical_and(df["subject"] == sub, df['experiment']==exp)]
                    sessions=data[np.logical_and(data['montage']==mont,data['localization']==loc)]['session'].unique()

                    for sess in sessions: 
                        reader = CMLReader(sub, exp, sess, montage=mont, localization=loc)   #reader for loading CML data
                        evs = reader.load('task_events')
#                         word_evs = evs[evs['type']=='REC_WORD']
                        events.append(evs)
                        
    except:
        print("failed for sub"+sub)

In [46]:
evs_df = pd.concat(events, ignore_index=True)
evs_df[:50]

,eegoffset,eegfile,exp_version,experiment,intrusion,is_stim,item_name,item_num,list,montage,...,session,stim_list,stim_params,subject,type,phase,recog_resp,recog_rt,recognized,rejected
0,282349,R1001P_FR1_0_12Oct14_1034,1.02,FR1,-999,0,X,-999,-999,0.0,...,0,0,[],R1001P,SESS_START,NaN,NaN,NaN,NaN,NaN
1,314149,R1001P_FR1_0_12Oct14_1034,1.02,FR1,-999,0,X,-999,-999,0.0,...,0,0,[],R1001P,COUNTDOWN_START,NaN,NaN,NaN,NaN,NaN
2,319363,R1001P_FR1_0_12Oct14_1034,1.02,FR1,-999,0,X,-999,-999,0.0,...,0,0,[],R1001P,COUNTDOWN_END,NaN,NaN,NaN,NaN,NaN
3,321225,R1001P_FR1_0_12Oct14_1034,1.02,FR1,-999,0,CHEST,-1,-1,0.0,...,0,0,[],R1001P,PRACTICE_WORD,NaN,NaN,NaN,NaN,NaN
4,322476,R1001P_FR1_0_12Oct14_1034,1.02,FR1,-999,0,RING,-1,-1,0.0,...,0,0,[],R1001P,PRACTICE_WORD,NaN,NaN,NaN,NaN,NaN
5,323675,R1001P_FR1_0_12Oct14_1034,1.02,FR1,-999,0,COTTON,-1,-1,0.0,...,0,0,[],R1001P,PRACTICE_WORD,NaN,NaN,NaN,NaN,NaN
6,324867,R1001P_FR1_0_12Oct14_1034,1.02,FR1,-999,0,CAMEL,-1,-1,0.0,...,0,0,[],R1001P,PRACTICE_WORD,NaN,NaN,NaN,NaN,NaN
7,326059,R1001P_FR1_0_12Oct14_1034,1.02,FR1,-999,0,TONGUE,-1,-1,0.0,...,0,0,[],R1001P,PRACTICE_WORD,NaN,NaN,NaN,NaN,NaN
8,327285,R1001P_FR1_0_12Oct14_1034,1.02,FR1,-999,0,BEAM,-1,-1,0.0,...,0,0,[],R1001P,PRACTICE_WORD,NaN,NaN,NaN,NaN,NaN
9,328551,R1001P_FR1_0_12Oct14_1034,1.02,FR1,-999,0,FLOOD,-1,-1,0.0,...,0,0,[],R1001P,PRACTICE_WORD,NaN,NaN,NaN,NaN,NaN


In [47]:
lag_crp = []
evs_df = evs_df[(evs_df.subject != 'R1001P') & (evs_df.list >= 0)]
pb.pd_crp?
# evs_df.groupby(['subject']).apply(lambda x: pb.pd_crp(x, itemno_column='item_num', list_index=['subject', 'session', 'list'], lag_num=6, ))





# for sub in evs_df['subject'].unique():
# #     try: 
#         sub_df = evs_df[evs_df['subject']==sub]
#         lag = pb.pd_crp(sub_df, itemno_column='item_num', list_index=['subject', 'session', 'list'], lag_num=6)
# #         lag['subject'] = pd.Series(sub, index = lag.index)
#         lag_crp.append(lag)
# #     except:
#         print("Failed for subject "+sub)
# lag_df = pd.concat(lag_crp)

Signature:
pb.pd_crp(
    df,
    lag_num=5,
    itemno_column='itemno',
    list_index=['subject', 'session', 'list'],
)
Docstring: Expects as input a dataframe (df) for one subject
File:      ~/analysis/mental_health/Mental_health/pandas_to_pybeh.py
Type:      function
